In [1]:
import pandas as pd
import numpy as np
import pybaseball as pyb
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt, atan2, degrees, pi, atan
from scipy.stats import percentileofscore

In [2]:
statcast_data = pyb.statcast(start_dt='2023-03-30', end_dt='2023-10-01')

This is a large query, it may take a moment to complete


/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|█████████████████████████████████████████| 186/186 [02:32<00:00,  1.22it/s]


In [3]:
fastballs = statcast_data[statcast_data['pitch_type'].isin(['FF'])]

In [4]:
fastballs.dropna(subset=['vy0'], inplace=True)

/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [5]:
fastballs.dropna(subset=['release_extension'], inplace=True)

In [6]:
def calculate_VRA(vy0, ay, release_extension, vz0, az):
    vy_s = -np.sqrt(vy0**2 - 2 * ay * (60.5 - release_extension - 50))
    t_s = (vy_s - vy0) / ay
    vz_s = vz0 - az * t_s
    VRA = -np.arctan(vz_s / vy_s) * (180 / np.pi)
    return VRA

def calculate_HRA(vy0, ay, release_extension, vx0, ax):
    vy_s = -np.sqrt(vy0**2 - 2 * ay * (60.5 - release_extension - 50))
    t_s = (vy_s - vy0) / ay
    vx_s = vx0 - ax * t_s
    HRA = -np.arctan(vx_s / vy_s) * (180 / np.pi)
    return HRA

fastballs['VRA'] = fastballs.apply(lambda x: calculate_VRA(x['vy0'], x['ay'], x['release_extension'], x['vz0'], x['az']), axis=1)
fastballs['HRA'] = fastballs.apply(lambda x: calculate_HRA(x['vy0'], x['ay'], x['release_extension'], x['vx0'], x['ax']), axis=1)

/var/folders/bf/7kjr62fs6wq3x9hlt0xsr0w40000gn/T/ipykernel_32718/2112343944.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastballs['VRA'] = fastballs.apply(lambda x: calculate_VRA(x['vy0'], x['ay'], x['release_extension'], x['vz0'], x['az']), axis=1)
/var/folders/bf/7kjr62fs6wq3x9hlt0xsr0w40000gn/T/ipykernel_32718/2112343944.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastballs['HRA'] = fastballs.apply(lambda x: calculate_HRA(x['vy0'], x['ay'], x['release_extension'], x['vx0'], x['ax']), ax

In [7]:
fastballs['VRA'] = pd.to_numeric(fastballs['VRA'], errors='coerce')
fastballs['HRA'] = pd.to_numeric(fastballs['HRA'], errors='coerce')

/var/folders/bf/7kjr62fs6wq3x9hlt0xsr0w40000gn/T/ipykernel_32718/935682892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastballs['VRA'] = pd.to_numeric(fastballs['VRA'], errors='coerce')
/var/folders/bf/7kjr62fs6wq3x9hlt0xsr0w40000gn/T/ipykernel_32718/935682892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastballs['HRA'] = pd.to_numeric(fastballs['HRA'], errors='coerce')


In [8]:
filtered_fastballs = fastballs.groupby(['game_date', 'player_name']).filter(lambda x: len(x) >= 25)

pitcher_game_dates = filtered_fastballs.groupby('player_name')['game_date'].nunique()

qualified_pitchers = pitcher_game_dates[pitcher_game_dates >= 10].index

qualified_fastballs = filtered_fastballs[filtered_fastballs['player_name'].isin(qualified_pitchers)]

qualified_fastballs['VRA_std_dev'] = qualified_fastballs.groupby(['game_date', 'player_name'])['VRA'].transform('std')

pitcher_stats = qualified_fastballs.groupby('player_name')['VRA_std_dev'].agg(['max', 'min']).reset_index()

pitcher_stats['diff_max_min'] = pitcher_stats['max'] - pitcher_stats['min']


/var/folders/bf/7kjr62fs6wq3x9hlt0xsr0w40000gn/T/ipykernel_32718/4165265084.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified_fastballs['VRA_std_dev'] = qualified_fastballs.groupby(['game_date', 'player_name'])['VRA'].transform('std')


In [9]:
sorted_pitcher_stats = pitcher_stats.sort_values('diff_max_min', ascending=False)

In [11]:
mean_std_dev_per_pitcher = qualified_fastballs.groupby('player_name')['VRA_std_dev'].mean().reset_index()

In [12]:
new_df = sorted_pitcher_stats.merge(mean_std_dev_per_pitcher, on='player_name')

In [13]:
median_vra_std_dev = new_df['diff_max_min'].median()
print("Median of diff_max_min in new_df:", median_vra_std_dev)


Median of diff_max_min in new_df: 0.39664901747962333


In [14]:
kirby_data = qualified_fastballs[qualified_fastballs['player_name'] == 'Kirby, George']

kirby_grouped = kirby_data.groupby('game_date')['VRA_std_dev'].mean()

vra_std_dev_values = kirby_grouped.values

window_size = 2  # Adjust window size as needed

moving_averages = np.convolve(vra_std_dev_values, np.ones(window_size)/window_size, mode='valid')

print(moving_averages)


[0.74944581 0.69111439 0.59672513 0.67760339 0.7309817  0.65010568
 0.69715683 0.76486561 0.65497299 0.56628291 0.57170631 0.62494174
 0.65678368 0.73687012 0.87766006 0.75146994 0.6388492  0.69446464
 0.73852342 0.71698988 0.7333343  0.76021599 0.78163044 0.76557624
 0.65260004]


In [15]:
filtered_fastballs = qualified_fastballs.groupby(['game_date', 'player_name']).filter(lambda x: len(x) >= 25)

pitcher_game_dates = filtered_fastballs.groupby('player_name')['game_date'].nunique()

qualified_pitchers = pitcher_game_dates[pitcher_game_dates >= 10].index

qualified_fastballs = filtered_fastballs[filtered_fastballs['player_name'].isin(qualified_pitchers)]

grouped_data = qualified_fastballs.groupby(['game_date', 'player_name'])['VRA_std_dev'].mean().reset_index()

grouped_data['rolling_avg_2game'] = grouped_data.groupby('player_name')['VRA_std_dev'].shift(-1)
grouped_data['rolling_avg_2game'] = (grouped_data['rolling_avg_2game'] + grouped_data['VRA_std_dev']) / 2

pitcher_stats = grouped_data.groupby('player_name')['rolling_avg_2game'].agg(['max', 'min']).reset_index()


In [16]:
pitcher_stats['diff_max_min'] = pitcher_stats['max'] - pitcher_stats['min']

In [17]:
george_kirby_stats = pitcher_stats[pitcher_stats['player_name'] == 'Kirby, George']

In [18]:
median_vra_std_dev_rolling = pitcher_stats['diff_max_min'].median()
print("Median of diff_max_min in new_df:", median_vra_std_dev_rolling)


Median of diff_max_min in new_df: 0.26466340127654836


In [19]:
new_df_rolling = pitcher_stats.merge(mean_std_dev_per_pitcher, on='player_name')

In [20]:
new_df_rolling = new_df_rolling.sort_values('diff_max_min', ascending=False)